In [1]:
#https://stackoverflow.com/questions/23566515/multiplication-of-1d-arrays-in-numpy

In [2]:
import numpy as np
import copy

In [3]:
def f(x):
    a=2*x[0]**2+ 3*x[1]**2 -3*x[0]*x[1] +2*x[0] -4*x[1]
    return a

In [4]:
def grad(x):
    a=np.array([4*x[0]-3*x[1]+2,6*x[1]-3*x[0]-4])
    return(a)

In [5]:
def normgrad(x):
    a=grad(x)
    b=(a[0]**2 + a[1]**2)**(.5)
    return b

# Line Search

In [6]:
alpha=0.5
def phi(x,alpha,d):
    ld=d*alpha
    y=x+ld
    phia=f(y)
    return phia

In [7]:
alpha1=0
alpha2=np.inf
alpha0=0.5 #(given in the question as x_0 = 0.5)
rho=0.1 #( do )
t=3
l=[]
l.append(alpha)

In [8]:
def goldstein(alpha1,alpha2,x,alpha,d,t,rho):
    cond1=True
    cond2=False
    val=phi(x,alpha,d) 
    while cond2==False:
        r=alpha+1
        if( phi(x,alpha,d)  > (f(x) +((grad(x)).dot(d))*rho*alpha)):
            cond1= False
        while cond1==False:
            r=alpha
            alpha2=alpha
            alpha= (alpha1+alpha2)/2
            l.append(alpha)
            if (phi(x,alpha,d)  <= (f(x) +((grad(x)).dot(d))*rho*alpha)):
                cond1=True
        if (phi(x,alpha,d)  >= f(x) +((grad(x)).dot(d))*(1-rho)*alpha): 
            cond2= True
        else:
            r=alpha
            alpha1=alpha
            if alpha2<np.inf:
                alpha=(alpha1+alpha2)/2
                l.append(alpha)
            else:
                alpha=t*alpha
                l.append(alpha)
        #if (abs(r-alpha)< 10**(-4) ):
        #    break
    return alpha

# DFP

In [9]:
def p(lam,d):
    a=lam*d
    return a

In [10]:
def graddiff(y2,y1):
    a=grad(y2)
    b=grad(y1)
    c=b-a
    return c

In [11]:
#https://stackoverflow.com/questions/3890621/how-does-multiplication-differ-for-numpy-matrix-vs-array-classes

In [12]:
def identity():
    a=np.matrix("1,0;0,1")
    return a

In [13]:
D1=np.identity(2)

# Actual algorithm

In [14]:
def dfpfull(epsilon,xold,D):
    listx=[]
    listy=[]
    listdirec=[]
    k=1
    j=1
    listx.append(xold)
    y1=copy.deepcopy(xold)
    listy.append(xold)
    cond=True
    while cond==True:
        cond,k,a1,a2,a3,a4,a5=DFPone(cond,epsilon,listy[-1],listx[-1],j,k,listx,listy,listdirec,D)
        listy.append(a5)
    return (k,a1,a2,a3,a4)

In [15]:
def DFPone(cond,epsilon,y,xold,j,k,listx,listy,listdirec,D): #input y is a numpy array
    #y1=copy.deepcopy(x1)
    if normgrad(y) <epsilon :
        cond=False
        listy.pop()
        return (cond,k,j,listx,listy,listdirec,y)
    elif j == 1:
        #listy.append(y)
        y1=copy.deepcopy(y) 
        g=grad(y1)
        d1= -D.dot(g)
        listdirec.append(d1)
        lam=goldstein(0,np.inf,y1,0.5,d1,3,.1)
        ld=d1*lam
        y2= y1 + ld
        listy.append(y2) 
        D= dfptwo(ld,y1,y2,D)
        j =j+1
        rr=DFPone(cond,epsilon,y2,xold,j,k,listx,listy,listdirec,D)
        return rr
    else:
        y2= copy.deepcopy(y)
        g=grad(y2)
        d2= -D.dot(g)
        listdirec.append(d2)
        lam=goldstein(0,np.inf,y2,0.5,d2,3,.1) 
        ld=d2*lam
        y3= y2 + ld
        #listy.append(y3) 
        listx.append(y3) 
        k =k+1
        j=1
        y1=y3
        D=np.identity(2)
    return (cond,k,j,listx,listy,listdirec,y1)

In [16]:
def dfptwo(ld,y1,y2,D):
    #a=np.matrix(ld) #dimension is 1x2
    a=ld # numpy array 
    q= graddiff(y2,y1) # numpy array
    #q=np.matrix(diff) #dimension is 1x2
    ppt=np.dot(a[:,None],a[None,:])
    #print(ppt)
    ptp=a.dot(a)
    qqt=np.dot(q[:,None],q[None,:])
    term2=ppt/ptp
    term3=(D.dot(qqt.dot(D)))/(q.dot(D.dot(q)))
    final=D + term2 - term3 
    return final

# Outputs

In [17]:
full=dfpfull(.000001,np.array([0,0]),D=np.identity(2))

In [18]:
full

(6,
 1,
 [array([0, 0]),
  array([-0.01923077,  0.65384615]),
  array([2.94035629e-16, 6.68269231e-01]),
  array([4.62278107e-05, 6.66697485e-01]),
  array([1.54326015e-16, 6.66662814e-01]),
  array([-1.11124545e-07,  6.66666593e-01])],
 [array([0, 0]),
  array([-0.25,  0.5 ]),
  array([-0.01923077,  0.65384615]),
  array([1.38777878e-16, 6.63461538e-01]),
  array([2.94035629e-16, 6.68269231e-01]),
  array([6.00961538e-04, 6.67067308e-01]),
  array([4.62278107e-05, 6.66697485e-01]),
  array([3.22008045e-17, 6.66674371e-01]),
  array([1.54326015e-16, 6.66662814e-01]),
  array([-1.44461908e-06,  6.66665704e-01]),
  array([-1.11124545e-07,  6.66666593e-01])],
 [array([-2.,  4.]),
  array([0.46153846, 0.30769231]),
  array([0.03846154, 0.01923077]),
  array([6.21031004e-16, 1.92307692e-02]),
  array([ 0.00480769, -0.00961538]),
  array([-0.00110947, -0.00073964]),
  array([-9.24556213e-05, -4.62278107e-05]),
  array([ 4.88500841e-16, -4.62278107e-05]),
  array([-1.15569527e-05,  2.31139053

# FRG

In [19]:
def FRGfull(epsilon,xold):
    listx=[]
    listy=[]
    listdirec=[]
    k=1
    j=1
    d=-grad(xold)
    listx.append(xold)
    listy.append(xold)
    cond=True
    while cond==True:
        d=-grad(listy[-1])
        cond,k,a1,a2,a3,a4,a5=FRGone(cond,epsilon,listy[-1],j,k,listx,listy,listdirec,d)
        listy.append(a5)
    return (k,a1,a2,a3,a4)

In [20]:
def FRGone(cond,epsilon,y,j,k,listx,listy,listdirec,d):
    if  normgrad(y)< epsilon :
        cond =False
        listy.pop()
        return (cond,k,j,listx,listy,listdirec,y)
    elif j==1 :
        #listy.append(y)
        y1= copy.deepcopy(y)
        d1=d
        listdirec.append(d1)
        lam=goldstein(0,np.inf,y1,0.5,d1,3,.1)
        ld= d1*lam
        y2 = y1+ ld
        listy.append(y2)
        g=-grad(y2)
        term2= (normgrad(y2)/normgrad(y1))**2
        d2 = g + d1*term2 
        d=d2
        #listdirec.append(d2)
        j =j+1
        rr= FRGone(cond,epsilon,y2,j,k,listx,listy,listdirec,d)
        
        return rr
    else:
        y2 =copy.deepcopy(y)
        d2=d
        listdirec.append(d2)
        lam=goldstein(0,np.inf,y2,0.5,d2,3,.1)
        ld = d2*lam
        y3 = y2 +ld
        #listy.append(y3)
        listx.append(y3)
        y1 =y3
        k=k+1
        j=1
    
    return (cond,k,j,listx,listy,listdirec,y3)    
        

In [21]:
full=dfpfull(.0001,np.array([0,0]),D=np.identity(2))
rfull =FRGfull(.0001,np.array([0,0]))

In [22]:
full[4]

[array([-2.,  4.]),
 array([0.46153846, 0.30769231]),
 array([0.03846154, 0.01923077]),
 array([6.21031004e-16, 1.92307692e-02]),
 array([ 0.00480769, -0.00961538]),
 array([-0.00110947, -0.00073964]),
 array([-9.24556213e-05, -4.62278107e-05])]

In [23]:
rfull[4]

[array([-2,  4]),
 array([0.46875, 0.3125 ]),
 array([0.03125 , 0.015625]),
 array([0.        , 0.01953125]),
 array([-0.00048828,  0.00097656]),
 array([1.14440918e-04, 7.62939453e-05])]

In [24]:
full[3]

[array([0, 0]),
 array([-0.25,  0.5 ]),
 array([-0.01923077,  0.65384615]),
 array([1.38777878e-16, 6.63461538e-01]),
 array([2.94035629e-16, 6.68269231e-01]),
 array([6.00961538e-04, 6.67067308e-01]),
 array([4.62278107e-05, 6.66697485e-01]),
 array([3.22008045e-17, 6.66674371e-01])]

In [25]:
rfull[3]

[array([0, 0]),
 array([-0.25,  0.5 ]),
 array([-0.015625,  0.65625 ]),
 array([0.       , 0.6640625]),
 array([0.        , 0.66650391]),
 array([-6.10351562e-05,  6.66625977e-01]),
 array([-3.81469727e-06,  6.66664124e-01])]

In [26]:
rfull[3]

[array([0, 0]),
 array([-0.25,  0.5 ]),
 array([-0.015625,  0.65625 ]),
 array([0.       , 0.6640625]),
 array([0.        , 0.66650391]),
 array([-6.10351562e-05,  6.66625977e-01]),
 array([-3.81469727e-06,  6.66664124e-01])]